In [1]:
R.version.string


[1] "R version 4.4.2 (2024-10-31)"

# Part 1 Regression (50 Marks)

Around 1000 people were questioned in a [life and wellbeing survey](https://www.get-happier.com/) to build a model to predict happiness of an individual. You need to build regression models to optimally predict the variable in the survey dataset called 'happiness' based on any, or all, of the other survey question responses.

You have been provided with two datasets, ```regression_train.csv``` and ```regression_test.csv```. Using these datasets, you hope to build a model that can predict happiness level using the other variables. ```regression_train.csv``` comes with the ground-truth target label (i.e. happiness level) whereas `regression_test.csv` comes with independent variables (input information) only.

On the order of around 70 survey questions have been converted into predictor variables that can be used to predict happiness. We do not list all the predictor names here, but their names given in the data header can clearly be linked to the survey questions. E.g. the predictor variable 'iDontFeelParticularlyPleasedWithTheWayIAm' corresponds to the survey question 'I don’t feel particularly pleased with the way I am.'

**PLEASE NOTE THAT THE USE OF LIBRARIES ARE PROHIBITED IN THESE QUESTIONS UNLESS STATED OTHERWISE, ANSWERS USING LIBRARIES WILL RECEIVE 0 MARKS**

## Question 1 (NO LIBRARIES ALLOWED) (4 Mark)
Please load the ```regression_train.csv``` and fit a [$\textbf{multiple linear regression model}$](https://en.wikipedia.org/wiki/Linear_regression) with 'happiness' being the target variable. According to the summary table, which predictors do you think are possibly associated with the target variable (use the significance level of 0.01), and which are the **Top 5** strongest predictors? Please write an R script to automatically fetch and print this information.

**NOTE**: Manually doing the above tasks will result in 0 marks.

In [2]:
install.packages("tidyverse")

library(tidyverse)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
# Load data
train_data <- read.csv("/content/regression_train.csv")

# Fit the linear regression model
model <- lm(happiness ~ ., data = train_data)

# Get p values from the model
p_values <- summary(model)$coefficients[, "Pr(>|t|)"]

# Filter predictors with p-values <= 0.01
significant_predictors <- names(p_values[p_values <= 0.01])

# Sort predictors by p-value and select the top 5
top_5_predictors <- names(p_values[order(p_values)][1:5])

# Print the top 5 strongest predictors
cat("\nTop 5 Strongest Predictors:\n")
print(top_5_predictors)



Top 5 Strongest Predictors:
[1] "income80k - 120k" "income50k - 80k"  "income200k above" "income20k - 50k" 
[5] "income15k - 20k" 


## Question 2 (2 Mark)
[**R squared**](https://en.wikipedia.org/wiki/Coefficient_of_determination) from the summary table reflects that the full model doesn't fit the training dataset well; thus, you try to quantify the error between the values of the ground-truth and those of the model prediction. You want to write a function to predict 'happiness' with the given dataset and calculate the [root mean squared error (rMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation) between the model predictions and the ground truths. Please test this function on the full model and the training dataset.

In [ ]:
# Load data
train_data <- read.csv("/content/regression_train.csv")

# Perform a full model for multiple linear regression
model <- lm(happiness ~ ., data = train_data)

calculate_rmse <- function(model, data) {
  # Predict 'happiness' values using the model
  predictions <- predict(model, newdata = data)

  # Calculate RMSE
  rmse <- sqrt(mean((data$happiness - predictions)^2))
  # return rmse
  return(rmse)
}

# Call function calculate rmse
full_model_rmse <- calculate_rmse(model, train_data)

cat("Root Mean Squared Error (RMSE) on the training dataset for the full model: ", full_model_rmse, "\n")


Root Mean Squared Error (RMSE) on the training dataset for the full model:  6.771948 


## Question 3 (2 Marks)
You find the full model complicated and try to reduce the complexity by performing [bidirectional stepwise regression](https://en.wikipedia.org/wiki/Stepwise_regression) with [BIC](https://en.wikipedia.org/wiki/Bayesian_information_criterion).

Calculate the **rMSE** of this new model with the function that you implemented previously. Is there anything you find unusual? Explain your findings in 100 words.

In [ ]:
# Load data
train_data <- read.csv("/content/regression_train.csv")

# Set Full Model
fullmod <- lm(happiness ~ ., train_data)

# Perform BIC
bic= step(fullmod, trace = 0, k = log(nrow(train_data)), direction = "both")

# Make predictions using the bic model
predicted_values <- predict(bic, newdata = train_data)

# Calculate the residuals
residuals <- train_data$happiness - predicted_values

# Calculate the mean squared error (MSE)
mse <- mean(residuals^2)

# Calculate the root mean squared error (RMSE)
rmse <- sqrt(mse)

# Print the RMSE
cat("Root Mean Squared Error (RMSE):", rmse, "\n")


Root Mean Squared Error (RMSE): 7.461174 


### Answer
The RMSE for the full model is 6.77. When we simplified the model using BIC, the RMSE increased slightly to 7.46, indicating that the simplified model's predictive performance on the training data was slightly worse. The unusual outcome suggests that the full model, including all predictors, may reveal important correlations, making it more effective. The result contradicts the popular idea that fewer predictors lead to a better model. However, to ensure the full model is genuinely better,  we must test it on an independent dataset to guarantee its generalization and predictive accuracy beyond the training data.


## Question 4 (2 Mark)
Although stepwise regression has reduced the model complexity significantly, the model still contains a lot of variables that we want to remove. Therefore, you are interested in lightweight linear regression models with ONLY TWO predictors. Write a script to automatically find the best lightweight model which corresponds to the model with the least **rMSE** on the training dataset. Compare the **rMSE** of the best lightweight model with the **rMSE** of the full model - ```lm.fit``` - that you built previously. Give an explanation for these results based on consideration of the predictors involved.

In [ ]:
# Function to calculate RMSE
calculate_rmse <- function(model, data) {
  predictions <- predict(model, data)
  rmse <- sqrt(mean((data$happiness - predictions)^2))
  return(rmse)
}

# Load data
data <- read.csv("/content/regression_train.csv")

# Initialize variables
best_model <- NULL
best_rmse <- Inf
best_predictors <- NULL

# Get the list of predictor
predictors <- colnames(data)[!colnames(data) %in% "happiness"]

# Iterate through all combinations of two predictors
for (i in 1:(length(predictors) - 1)) {
  for (j in (i + 1):length(predictors)) {
    # Get the two predictors for this iteration
    selected_predictors <- c(predictors[i], predictors[j])

    # Create the formula for the model
    formula <- as.formula(paste("happiness ~", paste(selected_predictors, collapse = " + ")))

    # Fit the model with two predictors
    model <- lm(formula, data)

    # Calculate the RMSE for this model
    rmse <- calculate_rmse(model, data)

    # Check if this model has a lower RMSE
    if (rmse < best_rmse) {
      best_rmse <- rmse
      best_model <- model
      # Store the selected predictors
      best_predictors <- selected_predictors
    }
  }
}

# Calculate the RMSE for the full model
full_model_rmse <- calculate_rmse(lm(happiness ~ ., data), data)

cat("RMSE for the best lightweight model: ", best_rmse, "\n")
cat("RMSE for the full model: ", full_model_rmse, "\n")

# Print the best two predictors
cat("The best two predictors for the lightweight model are: ", best_predictors[1], " and ", best_predictors[2], "\n")

# Compare the results
if (best_rmse < full_model_rmse) {
  cat("The best lightweight model has a lower RMSE, indicating better performance with just two predictors.\n")
} else {
  cat("The full model performs better in terms of RMSE, suggesting that including more predictors provides a better fit to the data.\n")
}


RMSE for the best lightweight model:  7.948134 
RMSE for the full model:  6.771948 
The best two predictors for the lightweight model are:  income  and  alwaysStressed 
The full model performs better in terms of RMSE, suggesting that including more predictors provides a better fit to the data.


### ANSWER (TEXT)

Lightmodel uses only "income" and "alwaysStressed" as predictors, which simplifies the model but reduces accuracy (RMSE 7.95). In contrast, a complex model with all predictors performs better (RMSE 6.77) because it considers more factors influencing happiness. The simpler model sacrifices accuracy for simplicity by omitting important factors. However, it's essential to find the right balance in the number of predictors. Too many predictors can lead to overfitting, where the model fits the training data perfectly but needs help with new data. It is important to balance the number of predictors used in predictions to prevent overfitting and improve generalization.

## Question 5 (Libraries are allowed) (40 Marks)
As a Data Scientist, one of the key tasks is to build models $\textbf{most appropriate/closest}$ to the truth; thus, modelling will not be limited to the aforementioned steps in this assignment. To simulate for a realistic modelling process, this question will be in the form of a [Kaggle competition](https://www.kaggle.com/t/a83b5384ae9849848356e0c1966771ec) among students to find out who has the best model.

Thus, you **will be graded** by the **rMSE** performance of your model, the better your model, the higher your score. Additionally, you need to describe/document your thought process in this model building process, this is akin to showing your working properly for the mathematic sections. If you don't clearly document the reasonings behind the model you use, we will have to make some deductions on your scores.

This is the [video tutorial](https://www.youtube.com/watch?v=rkXc25Uvyl4) on how to join any Kaggle competition.

When you optimize your model's performance, you can use any supervised model that you know and feature selection might be a big help as well. [Check the non-exhaustive set of R functions relevant to this unit](https://lms.monash.edu/mod/resource/view.php?id=12098821) for ideas for different models to try.

$\textbf{Note}$ Please make sure that we can install the libraries that you use in this part, the code structure can be:

```install.packages("some package", repos='http://cran.us.r-project.org')```

```library("some package")```

Remember that if we cannot run your code, we will have to give you a deduction. Our suggestion is for you to use the standard ```R version 3.6.1```

You also need to name your final model ``fin.mod`` so we can run a check to find out your performance. A good test for your understanding would be to set the previous $\textbf{BIC model}$ to be the final model to check if your code works perfectly.

# Explanation :

First, I used two techniques, Lasso and Boruta, to identify the predictors that truly affect happiness. I chose predictors that can either increase or decrease happiness, making sure to include both positive and negative correlations. This approach considers various factors influencing our happiness.

I chose the Random Forest algorithm due to its robustness and adaptability. Random Forest is a powerful tool that can handle multiple predictors, whether they are categorical (using one-hot encoding) or numerical. It can even handle predictors that have high collinearity. One of the unique strengths of Random Forest is that it keeps the model focused on complex details that could lead to overfitting. To ensure data integrity, I set a random seed, which promotes reproducibility when training the Random Forest model.

To fine-tune the model, I explored various hyperparameters, including ntree (the number of trees in the forest), mtry (the number of features considered at each split), nodesize (the minimum number of observations in terminal nodes), and minsplit (the minimum number of data points required to do a split). I also used cross-validation to evaluate the model's performance under different settings through repeated testing. I optimized my model by minimizing the Root Mean Square Error (RMSE) and quantifying the difference between predicted and actual happiness levels in training data. I compared the average RMSE, calculated as the mean of RMSE values across all cross-validation folds, with the best RMSE to ensure consistent performance. After several loops to find the best parameter, I will use the best model to predict test data.

Furthermore, I used a learning curve to visualize the model's performance with the training data. It helps assess the model's tendency toward overfitting or underfitting, ensuring its ability to generalize well to unseen data.

In [ ]:
install.packages('randomForest')
install.packages('caret')


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [55]:
# Load libraries
library(caret)
library(randomForest)

# Load training data
train_data <- read.csv("/content/regression_train.csv")

# Define predictor variables
predictor_variables <- c("alwaysAnxious", "alwaysStressed", "alwaysDepressed",
                         "iDontHaveFunWithOtherPeople", "alwaysHaveFun",
                         "iUsuallyHaveAGoodInfluenceOnEvents", "iLaughALot",
                         "alwaysLoveAndCareForYourself","iAlwaysHaveACheerfulEffectOnOthers")

# Convert income to factor
train_data$income <- factor(train_data$income)

# Create one-hot encoder
dummy_encoder <- dummyVars(~ income, data = train_data)
encoded_train_income <- predict(dummy_encoder, newdata = train_data)

# Combine predictors and encoded income for training data
X_train <- cbind(train_data[, predictor_variables], encoded_train_income)

# Define the target variable
y_train <- train_data$happiness

# Train the Random Forest model
set.seed(123)
rf_model <- randomForest(
  x = X_train,
  y = y_train,
  ntree = 500,
  importance = TRUE
)

# Load test data
test_data <- read.csv("/content/regression_test.csv")

test_data$income <- factor(test_data$income, levels = levels(train_data$income))

encoded_test_income <- tryCatch({
  predict(dummy_encoder, newdata = test_data)
}, error = function(e) {
  encoded_test_income <- predict(dummy_encoder, newdata = test_data, na.action = na.pass)
  missing_levels <- setdiff(colnames(encoded_train_income), colnames(encoded_test_income))
  for (lvl in missing_levels) {
    encoded_test_income[, lvl] <- 0
  }
  encoded_test_income <- encoded_test_income[, colnames(encoded_train_income), drop = FALSE]
  encoded_test_income
})

X_test <- cbind(test_data[, predictor_variables], encoded_test_income)

pred.label <- predict(rf_model, X_test)

# Save predictions to a CSV file
write.csv(
  data.frame("RowIndex" = seq(1, length(pred.label)), "Prediction" = pred.label),
  "RegressionPredictLabel.csv",
  row.names = FALSE
)


## RMSE Result
The private leaderboard is calculated with approximately **71%** of the test data.

The Random Forest approach achieved an RMSE of **3.90118** on the public leaderboard and **5.47298** on the private leaderboard in Kaggle.

# Part 2 Classification (50 Marks)

Around 1000 people were questioned in a [life and wellbeing survey](https://www.get-happier.com/) to build a model to predict happiness of an individual, but this time we want to predict a categorical score for perfect mental health, rather than a continuous score. You need to build 5-class classification models to optimally predict the variable in the survey dataset called 'perfectMentalHealth' based on any, or all, of the other survey question responses.

You have been provided with two datasets, ```classification_train.csv``` and ```classification_test.csv```. Using these datasets, you hope to build a model that can predict 'perfectMentalHealth' using the other variables. ```classification_train.csv``` comes with the ground-truth target label (i.e. 'perfectMentalHealth' happiness classes) whereas `classification_test.csv` comes with independent variables (input information) only.

On the order of around 70 survey questions have been converted into predictor variables that can be used to predict 'perfectMentalHealth'. We do not list all the predictor names here, but their names given in the data header can clearly be linked to the survey questions. E.g. the predictor variable 'iDontFeelParticularlyPleasedWithTheWayIAm' corresponds to the survey question 'I don’t feel particularly pleased with the way I am.'

This question will also be in the form of a [Kaggle competition](https://www.kaggle.com/t/9288c3d1ae9648d88b8ff809e22d44d2) among students to find out who has the best model.

# Explanation :

The goal was to build a robust model that could predict "perfect mental health" at a high level of accuracy. To achieve this, I utilised a feature selection technique known as the Kruskal-Wallis test. This non-parametric statistical test is particularly suitable for evaluating the relationship between an ordinal or continuous predictor variable and a categorical target variable. The choice of the Kruskal-Wallis test was motivated by several factors:

**Suitability for Ordinal Data**:

All predictors in the dataset were ordinal, and Kruskal-Wallis does not assume normality of data, making it a natural fit for such predictors.

**Capturing Non-linear Relationships**:

Unlike correlation-based methods, Kruskal-Wallis is capable of detecting non-linear associations between predictors and the target variable.

**Reduction of Dimensionality**:

By filtering out predictors with insignificant relationships to the target variable (p-value ≥ 0.05), the test helped focus the model on the most meaningful features, reducing overfitting and improving interpretability.


In the model selection process, I considered several other algorithms, including support vector machine, XGBoost, GBM, and logistic regression. After a thorough evaluation, I chose the Random Forest algorithm due to its performance and ability to handle predictors that are closely related without making the model overly complex, thus reducing the risk of overfitting. To maintain the integrity, I used a random seed for reproducibility.


In [3]:
install.packages("randomForest")
install.packages("caret")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




In [22]:
# Load libraries
library(randomForest)
library(caret)

# Load training and test data
train_data <- read.csv("/content/classification_train.csv")
test_data <- read.csv("/content/classification_test.csv")

# Define all predictor variables (all columns except the target variable)
predictors <- setdiff(names(train_data), "perfectMentalHealth")

# Define target variable
target_variable <- "perfectMentalHealth"

# Convert the target variable to a factor
train_data[, target_variable] <- as.factor(train_data[, target_variable])

# Initialize an empty data frame for test results
kw_results <- data.frame(Predictor = character(0), P_Value = numeric(0), stringsAsFactors = FALSE)

# Perform Kruskal-Wallis test for each predictor
for (predictor in predictors) {
  if (any(is.na(train_data[[predictor]]))) next

  # Perform Kruskal-Wallis test
  kw_test <- kruskal.test(as.formula(paste(predictor, "~", target_variable)), data = train_data)
  p_value <- kw_test$p.value

  # Store results
  kw_results <- rbind(kw_results, data.frame(Predictor = predictor, P_Value = p_value))
}

# Sort predictors by their p-value
kw_results <- kw_results[order(kw_results$P_Value), ]
print("Kruskal-Wallis Test Results:")
print(kw_results)

# Filter predictors with p-value < 0.05
significant_predictors <- kw_results$Predictor[kw_results$P_Value < 0.05]
cat("Significant predictors (p-value < 0.05):", paste(significant_predictors, collapse = ", "), "\n")

# Use only significant predictors for Random Forest
if (length(significant_predictors) > 0) {
  predictors <- significant_predictors
} else {
  cat("No significant predictors found! Using all predictors.\n")
}

# Split the training data into training and validation sets
set.seed(123)
train_indices <- createDataPartition(train_data[[target_variable]], p = 0.8, list = FALSE)
train_subset <- train_data[train_indices, ]
val_subset <- train_data[-train_indices, ]

# Train a Random Forest model with the selected predictors
set.seed(123)
rf_model <- randomForest(
  x = train_subset[, predictors],
  y = train_subset[[target_variable]],
  ntree = 500,
  mtry = 2,
  importance = TRUE
)

# Save the trained model to an RDS file
saveRDS(rf_model, "rf_model_kw_selected.rds")
cat("Random Forest model saved to 'rf_model_kw_selected.rds'\n")

# Make predictions on the test data
test_predictions <- predict(rf_model, newdata = test_data[, predictors])

# Save predictions to a CSV file
write.csv(
  data.frame("RowIndex" = seq(1, nrow(test_data)), "Prediction" = test_predictions),
  "RandomForestClassificationPredictions.csv",
  row.names = FALSE
)
cat("Test predictions saved to 'RandomForestClassificationPredictions.csv'\n")


[1] "Kruskal-Wallis Test Results:"
                                                         Predictor      P_Value
19                                    alwaysLoveAndCareForYourself 1.951199e-31
40                                                      lifeIsGood 1.102252e-25
16                                                   alwaysHaveFun 7.868241e-22
14                                                      alwaysCalm 1.611113e-19
27                         iAmWellSatisfiedAboutEverythingInMyLife 1.953604e-17
33 alwaysEngageInPreparingAndUsingYourSkillsAndTalentsInOrderToGai 2.003534e-16
37                                    iFeelThatLifeIsVeryRewarding 2.102050e-15
12                                                  alwaysStressed 6.509653e-14
30                      iFeelThatIAmNotEspeciallyInControlOfMyLife 2.248593e-12
20                                     extremelyGoodAbilityToSense 6.119040e-09
23                                      iRarelyWakeUpFeelingRested 6.979445e-09
31   

During my initial approach, I struggled to achieve higher leaderboard rankings.

However, by revisiting this problem and applying the refined methodology outlined here, I was able to achieve:

Private leaderboard score (71% of the data): **0.40407**,
corresponding to Rank **3/167** on the private leaderboard.


Public leaderboard score (29% of the data): **0.66246**, corresponding to Rank **65/167** on the public leaderboard.

